In [12]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# API 키 정보 로드
load_dotenv()

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangGraph-Memory")

LangSmith 추적을 시작합니다.
[프로젝트명]
LangGraph-Memory


In [ ]:
from langgraph.checkpoint.memory import MemorySaver
# from langgraph.checkpoint.sqlite import SqliteSaver
# from langgraph.checkpoint.postgres import PostgresSaver

# 메모리 저장소 생성
memory = MemorySaver()

In [31]:
from typing import Annotated
from typing_extensions import TypedDict
from langchain_ollama import ChatOllama
from langchain_teddynote.tools.tavily import TavilySearch
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


########## 1. 상태 정의 ##########
# 상태 정의
class State(TypedDict):
    # 메시지 목록 주석 추가
    messages: Annotated[list, add_messages]

########## 2. 도구 정의 및 바인딩 ##########
# 도구 초기화
tool = TavilySearch(max_results=3)
tools = [tool]
# LLM 초기화
llm = ChatOllama(model="llama3.1:8b")

########## 3. 노드 추가 ##########
# 챗봇 함수 정의
def chatbot(state: State):
    # 메시지 호출 및 반환
    return {"messages": [llm.invoke(state["messages"])]}

# 상태 그래프 생성
graph_builder = StateGraph(State)
# 챗봇 노드 추가
graph_builder.add_node("chatbot", chatbot)
# 도구 노드 생성 및 추가
tool_node = ToolNode(tools)
# 도구 노드 추가
graph_builder.add_node("tools", tool_node)
# 조건부 엣지
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

########## 4. 엣지 추가 ##########

# tools > chatbot
graph_builder.add_edge("tools", "chatbot")
# START > chatbot
graph_builder.add_edge(START, "chatbot")
# chatbot > END
graph_builder.add_edge("chatbot", END)

In [32]:
graph = graph_builder.compile(checkpointer=memory)

In [33]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    recursion_limit=10,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "1"},  # 스레드 ID 설정
)

In [34]:
# 첫 질문
question = (
    "내 이름은 `PETER` 입니다. 한국에 살고 있는 개발자에요. 만나서 반가워요."
)

for event in graph.stream({"messages": [("user", question)]}, config=config):
    for value in event.values():
        value["messages"][-1].pretty_print()

================================== Ai Message ==================================

안녕하세요! 반갑습니다! 한국에서生活하시는 PETER 님 인사해 봅니다. 어떤 프로젝트나 개발 공부를 하고 계신가요?


In [35]:
# 이어지는 질문
question = "내 이름이 뭐라고 했지?"

for event in graph.stream({"messages": [("user", question)]}, config=config):
    for value in event.values():
        value["messages"][-1].pretty_print()

================================== Ai Message ==================================

PETER 였어요! 

그럼, 잘 지내시나요? 한국의 개발자로서 어떤 프로젝트나 관심사에 집중하고 계신가요?


In [36]:
from langchain_core.runnables import RunnableConfig

question = "내 이름이 뭐라고 했지?"

config = RunnableConfig(
    recursion_limit=10,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "2"},  # 스레드 ID 설정
)

for event in graph.stream({"messages": [("user", question)]}, config=config):
    for value in event.values():
        value["messages"][-1].pretty_print()

================================== Ai Message ==================================

아무래도 저랑 처음 대화하는 거라 기억할 수 있는 정보는 없네요. 첫 메세지를 남겨주면 제가 기억하고 알려드릴게요!


### Snapshot

In [37]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    configurable={"thread_id": "1"},  # 스레드 ID 설정
)
# 그래프 상태 스냅샷 생성
snapshot = graph.get_state(config)
snapshot.values

{'messages': [HumanMessage(content='내 이름은 `PETER` 입니다. 한국에 살고 있는 개발자에요. 만나서 반가워요.', additional_kwargs={}, response_metadata={}, id='fc84a423-51cc-46f0-b9cd-1bc2eb625e9c'),
  AIMessage(content='안녕하세요! 반갑습니다! 한국에서生活하시는 PETER 님 인사해 봅니다. 어떤 프로젝트나 개발 공부를 하고 계신가요?', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-02-04T02:09:11.329607Z', 'done': True, 'done_reason': 'stop', 'total_duration': 48531233208, 'load_duration': 42450503208, 'prompt_eval_count': 36, 'prompt_eval_duration': 2984000000, 'eval_count': 36, 'eval_duration': 3095000000, 'message': {'role': 'assistant', 'content': '', 'images': None, 'tool_calls': None}}, id='run-9d86403f-1337-44ef-bd1e-56534ccd4aa4-0', usage_metadata={'input_tokens': 36, 'output_tokens': 36, 'total_tokens': 72}),
  HumanMessage(content='내 이름이 뭐라고 했지?', additional_kwargs={}, response_metadata={}, id='bd9cb0ff-57fd-413a-a204-b9f3a2a589a6'),
  AIMessage(content='PETER 였어요! \n\n그럼, 잘 지내시나요? 한국의 개발자로서 어떤 프로젝트나 관심사에 집중하고 계신가

In [38]:
snapshot.config

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1efe29d0-ee82-65e0-8004-ef285cac4c9c'}}

In [39]:
snapshot.next

()

In [40]:
from langchain_teddynote.messages import display_message_tree

# 메타데이터(tree 형태로 출력)
display_message_tree(snapshot.metadata)


    source: "loop"
    writes:
        chatbot:
            messages:
                index [0]
                    content: "PETER 였어요! 

그럼, 잘 지내시나요? 한국의 개발자로서 어떤 프로젝트나 관심사에 집중하고 계신가요?"
                    additional_kwargs: {}
                    response_metadata:
                        model: "llama3.1:8b"
                        created_at: "2025-02-04T02:09:25.955784Z"
                        done: True
                        done_reason: "stop"
                        total_duration: 3914054583
                        load_duration: 36112750
                        prompt_eval_count: 89
                        prompt_eval_duration: 387000000
                        eval_count: 37
                        eval_duration: 3158000000
                        message: {"role": "assistant", "content": "", "images": None, "tool_calls": None}
                    type: "ai"
                    name: None
                    id: "run-e245aaf2-2829-44d5-be9f-da148e30f7a0-0"
              